# 6 WorkFlow

### 6.1 Objetivo

Presentar un workflow/pipeline completo al que los estudiantes deberán enriquecer

#### 6.2  Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Type -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

In [ ]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Mounted at /content/.drive


Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria  kaggle de Python
*   Bajar el  **dataset_pequeno**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab
*   Bajar el **dataset_historico** al Google Drive y tambien al disco local de la virtual machine que esta corriendo Google Colab



In [ ]:
%%shell

mkdir -p "/content/.drive/My Drive/labo2"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/labo2" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets


webfiles="https://storage.googleapis.com/open-courses/austral2025-af91/"
destino_local="/content/datasets"
destino_bucket="/content/buckets/b1/datasets"


archivo="dataset_pequeno.csv"

if ! test -f $destino_bucket/$archivo; then
  wget  $webfiles/$archivo  -O $destino_bucket/$archivo
fi


if ! test -f $destino_local/$pequeno; then
  cp  $destino_bucket/$archivo  $destino_local/$archivo
fi

#-------

archivo="gerencial_competencia_2025.csv.gz"

if ! test -f $destino_bucket/$archivo; then
  wget  $webfiles/$archivo  -O $destino_bucket/$archivo
fi


if ! test -f $destino_local/$pequeno; then
  cp  $destino_bucket/$archivo  $destino_local/$archivo
fi


## 6.3  Workflow

## Inicializacion

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

limpio el ambiente de R

In [ ]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Sat Nov 22 09:23:54 PM 2025"

In [ ]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,660361,35.3,1454494,77.7,1454494,77.7
Vcells,1226541,9.4,8388608,64.0,1975136,15.1


In [ ]:
require("data.table")

if( !require("R.utils")) install.packages("R.utils")
require("R.utils")

if( !require("dplyr")) install.packages("dplyr")
require("dplyr")


Loading required package: data.table

Loading required package: R.utils

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘R.utils’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘R.oo’, ‘R.methodsS3’


Loading required package: R.utils

Loading required package: R.oo

Loading required package: R.methodsS3

R.methodsS3 v1.8.2 (2022-06-13 22:00:14 UTC) successfully loaded. See ?R.methodsS3 for help.

R.oo v1.27.1 (2025-05-02 21:00:05 UTC) successfully loaded. See ?R.oo for help.


Attaching package: ‘R.oo’


The following object is masked from ‘package:R.methodsS3’:

    throw


The following objects are masked from ‘package:methods’:

    getClasses, getMethods


The following objects are masked from ‘package:base’:

    attach, detach, load, save


R.utils v2.13.0 (2025-02-24 21:20:02 UTC) successfully loaded. See ?R.utils for help.




#### Parametros
Si es gerente, no cambie nada
<br>Si es Analista, cambie el nombre del dataset

In [ ]:
PARAM <- list()
PARAM$semilla_primigenia <- c(100151, 100237, 100291, 345689, 655883, 334561, 478453, 627269, 778889 ,819509)

PARAM$experimento <- 0001
PARAM$dataset <- "gerencial_competencia_2025.csv.gz"

#### Carpeta del Experimento

In [ ]:
# carpeta de trabajo

setwd("/content/buckets/b1/exp")
experimento_folder <- paste0("PCA", PARAM$experimento)
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

### 6.3.1   Preprocesamiento del dataset

#### 6.3.1.1  DT incorporar dataset

In [ ]:
# lectura del dataset
data <- fread(paste0("/content/datasets/", PARAM$data))
print(data)

        numero_de_cliente foto_mes internet cliente_edad cliente_antiguedad
                    <int>    <int>    <int>        <int>              <int>
     1:          29186441   202005        1           61                282
     2:          29187961   202005        1           61                312
     3:          29193101   202005        1           67                365
     4:          29193281   202005        1           54                158
     5:          29198891   202005        1           54                312
    ---                                                                    
273662:         187838971   202109        0           49                  1
273663:         187902361   202109        0           22                  1
273664:         187956461   202109        0           34                  1
273665:         187964701   202109        0           26                  1
273666:         188116871   202109        0           30                  1
        mren

#### 6.3.1.2  Análisis de Componentes Principales
Se intentan reducir la dimensionalidad del dataset para trabajar con menos variables

In [ ]:
#Verificar si existen filas que tengan algún nulo para eliminarlas (el PCA no permite nulos)

filas_con_na <- data %>%
  filter(if_any(everything(), is.na))

total_na <- nrow(filas_con_na)

# Conteo y porcentaje por clase
res_na <- filas_con_na %>%
  count(clase_ternaria) %>%
  mutate(porcentaje = 100 * n / total_na)

res_na

clase_ternaria,n,porcentaje
<chr>,<int>,<dbl>
,1758,5.907060
BAJA+1,416,1.397802
BAJA+2,357,1.199556
CONTINUA,27230,91.495581


In [ ]:
#Eliminar las filas que tienen algún valor nulo

dataset_sin_na <- data %>%
  filter(if_all(everything(), ~ !is.na(.x)))

dataset_sin_na

numero_de_cliente,foto_mes,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,mpasivos_margen,⋯,ccallcenter_transacciones,chomebanking_transacciones,ctrx_quarter,Master_status,Master_fechaalta,Master_mpagominimo,Visa_status,Visa_fechaalta,Visa_mpagominimo,clase_ternaria
<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<int>,<int>,<int>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<chr>
29186441,202005,1,61,282,2354.94,43761.59,1682.14,-139.02,712.60,⋯,0,20,47,0,6507,0.00,0,7671,1043.97,CONTINUA
29187961,202005,1,61,312,5066.31,47268.33,3875.62,-3536.03,4551.11,⋯,0,228,232,0,5939,2017.56,0,8090,7495.47,CONTINUA
29193101,202005,1,67,365,4809.42,37996.66,6982.73,-2962.24,1109.46,⋯,0,133,149,0,2026,2838.66,0,2026,1653.93,CONTINUA
29193281,202005,1,54,158,2979.52,88436.59,3350.01,-908.55,592.70,⋯,2,10,115,0,3125,844.56,0,3350,2029.29,CONTINUA
29198891,202005,1,54,312,3008.70,22567.32,273.68,-643.87,2975.52,⋯,0,124,32,0,7202,234.60,0,7242,1395.87,CONTINUA
29205441,202005,1,72,6,394.19,1607.93,159.49,-9.92,210.01,⋯,0,41,32,0,157,0.00,0,157,58.65,CONTINUA
29243631,202005,1,54,110,2043.10,32934.54,492.60,915.30,406.53,⋯,0,87,81,0,992,0.00,0,3036,1395.87,CONTINUA
29251231,202005,1,82,312,1071.17,80557.53,635.61,-51.43,422.75,⋯,0,91,58,0,7298,304.98,0,8215,105.57,CONTINUA
29257551,202005,1,44,187,1552.31,11131.46,390.96,8.91,981.16,⋯,1,92,55,0,3022,70.38,0,5632,1196.46,CONTINUA


In [ ]:
#Estandarizar todas las variables excepto "clase_ternaria" que es nuestra etiqueta,
#la variable identificadora "numero_de_cliente" y
#"foto_mes" que se va a utilizar como future adicional después de aplicar PCA

cols_excluir <- c("clase_ternaria", "numero_de_cliente", "foto_mes")

# Columnas que SÍ querés escalar
cols_escalar <- setdiff(colnames(dataset_sin_na), cols_excluir)

# 1. Escalar solo las columnas deseadas
dataset_scaled <- dataset_sin_na %>%
  mutate(across(all_of(cols_escalar), scale))

In [ ]:
#Cálculo de componentes principales

dataset_numeric <- dataset_scaled %>% select(all_of(cols_escalar))

pca <- prcomp(dataset_numeric, center = TRUE, scale. = TRUE)

In [ ]:
#Visualizar los resultados
summary(pca)

Importance of components:
                          PC1    PC2     PC3     PC4    PC5     PC6     PC7
Standard deviation     2.1154 1.7004 1.55259 1.40073 1.2801 1.25172 1.11990
Proportion of Variance 0.1543 0.0997 0.08312 0.06766 0.0565 0.05403 0.04325
Cumulative Proportion  0.1543 0.2540 0.33712 0.40478 0.4613 0.51531 0.55856
                           PC8     PC9    PC10    PC11    PC12    PC13    PC14
Standard deviation     1.04399 0.97936 0.96012 0.95555 0.94123 0.93412 0.91880
Proportion of Variance 0.03758 0.03307 0.03179 0.03149 0.03055 0.03009 0.02911
Cumulative Proportion  0.59614 0.62921 0.66100 0.69249 0.72304 0.75313 0.78224
                          PC15    PC16    PC17    PC18    PC19   PC20    PC21
Standard deviation     0.89401 0.84595 0.83312 0.81651 0.78457 0.7635 0.67243
Proportion of Variance 0.02756 0.02468 0.02393 0.02299 0.02123 0.0201 0.01559
Cumulative Proportion  0.80980 0.83447 0.85841 0.88140 0.90262 0.9227 0.93832
                          PC22    PC23    

In [ ]:
#Según el criterio de varianza acumulada bajo el cual se recomienda retener suficientes componentes para explicar
#entre el 80% y 90% de la varianza se decide dejar las primeras 15 componentes principales para el nuevo dataset

pca_df <- as.data.frame(pca$x[, 1:15])

meta_vars <- dataset_sin_na[, c("clase_ternaria", "numero_de_cliente", "foto_mes")]

dataset_pca <- cbind(meta_vars, pca_df)

dataset_pca

clase_ternaria,numero_de_cliente,foto_mes,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15
<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CONTINUA,29186441,202005,1.18760035,-1.59557626,-3.1494060,0.81892169,0.68046851,0.32943941,0.12289835,0.94073389,-0.95222632,-0.31995477,0.099012129,5.076757e-01,0.78637432,0.37954486,0.31895255
CONTINUA,29187961,202005,3.99154078,-2.68095953,-0.6438758,0.14095816,0.88054262,-0.42239732,-0.59030043,1.96805199,-1.13441174,0.15649556,0.254591828,-6.016703e-01,0.31523843,0.51905380,-1.03934675
CONTINUA,29193101,202005,2.25927548,-1.50968645,-0.3365537,0.73270152,1.68289375,0.44932668,-0.92965714,0.66557835,-0.45710013,-0.76929472,-0.020638185,-1.752474e+00,1.87414432,0.84466208,-0.48681184
CONTINUA,29193281,202005,0.91305631,-0.26590876,-0.7840070,0.67275262,1.16242919,0.58169242,-1.20236031,0.67473632,0.01054089,-0.94521525,-0.048268453,2.740263e-01,1.24345582,0.42727445,0.48909456
CONTINUA,29198891,202005,1.28776261,-1.38230765,-2.8397508,0.23021292,-0.58994206,0.06427492,0.23761360,1.91030965,-1.14035074,-0.15466670,0.184470349,5.056811e-02,0.58534609,-0.36839444,-0.55362252
CONTINUA,29205441,202005,-2.08455073,1.32669588,-0.1070258,0.14295075,0.16928138,0.32926282,-0.20557043,0.92959034,-0.52139873,-0.76132852,0.344704091,1.372932e-01,0.95883466,0.34054896,0.26174323
CONTINUA,29243631,202005,-0.33606306,0.21250515,-0.3646423,0.06356410,-0.93781151,0.92544614,-1.68677396,0.12907894,-0.95206379,-0.96715972,0.307300064,-8.521277e-02,1.07264007,-0.05371563,0.03359097
CONTINUA,29251231,202005,1.78988443,-2.04192588,-3.9041591,0.55910125,-0.75279604,0.60612561,-0.15776363,1.26993168,-0.91917551,-0.48597853,0.270121623,-1.401924e-01,0.95300119,-0.37254421,-0.14920105
CONTINUA,29257551,202005,-0.28326417,-0.34035517,-1.0453689,0.04024042,-0.56575146,0.14697363,-0.33206168,1.72731182,-0.68189813,-0.25962213,0.145904163,2.675606e-01,0.54143606,-0.05167896,0.10094129


In [ ]:
#Para poder cargar los resultados a Kaggle necesito que tenga las 16.551 filas
#Por eso vamos a imputar las filas que borramos e imputamos las PC=0

#Clientes del dfuture original
dfuture_original <- data[ foto_mes == 202109 ]

#Clientes del dfuture filtrado
cols_modelo <- setdiff(
    names(dataset_pca),
    c("clase_ternaria", "foto_mes")
)
dfuture_pca <- dataset_pca[ foto_mes == 202109, ..cols_modelo ]

#Buscar los clientes faltantes
faltan <- setdiff(dfuture_original$numero_de_cliente, dfuture_pca$numero_de_cliente)

# crear un data.table con PCs en cero
faltantes <- data.table(
   numero_de_cliente = faltan
)

#Agrego columnas de PC
pc_cols <- grep("^PC", names(dataset_pca), value = TRUE)
faltantes[, (pc_cols) := 0 ]

#Unir los dataset
dfuture_completo <- rbindlist(
  list(dfuture_pca, faltantes),
  use.names = TRUE,
  fill = TRUE
)

dfuture_completo[, foto_mes := 202109]

dfuture_completo

numero_de_cliente,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,foto_mes
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
29186441,3.64852445,-0.8403925,-2.6949387,-0.003597311,0.71047796,-0.56205593,0.91841909,0.263293510,0.06450335,0.33797529,-0.057137293,0.35202249,-0.52887263,0.07168419,-0.94373359,202109
29187961,3.67184777,-3.1712988,-0.8673135,0.586111884,0.92139605,-0.16426325,0.89201880,-0.314224491,-0.37405261,0.57562892,0.190614345,1.65164965,-0.91613637,0.06027089,0.56035659,202109
29193101,4.60568209,-0.8499590,0.4874577,1.689943647,1.99546994,1.56603963,0.21476938,-0.108062741,0.87606689,0.69446489,-0.543708010,-3.59292104,1.20499571,1.14514250,-1.02968820,202109
29193281,1.37668417,-0.6335746,-0.1699684,0.136589155,0.65562028,0.21888406,0.49149597,-0.772982811,0.21991224,-0.39048339,-0.222612692,-0.66374418,0.75342763,-0.41530237,0.04279998,202109
29198891,2.18142212,-1.2769720,-2.7271301,-0.009654375,-0.43681097,-0.25873261,1.65107408,0.271128910,-0.02282281,0.12284059,-0.129567664,0.45743393,-0.06519040,-0.98042334,-0.54190335,202109
29201701,-1.60385869,1.3306990,0.9186345,-0.113565190,0.24312440,0.19602781,0.45168216,-0.287005283,0.20008488,0.11487558,-0.130593250,-0.11898870,-0.15367021,0.01646758,-0.06863394,202109
29205441,-1.95883981,1.2417832,-0.5060453,2.479850482,0.56349473,-1.85576887,0.21772486,-0.699721282,2.24434241,0.48591687,8.914920931,-1.21146634,0.27604977,-0.27172870,-0.33781247,202109
29243631,0.07752157,0.5686526,-0.6886258,0.144666779,-0.95896552,0.97228099,-0.48163262,-0.476822106,0.07622836,0.31491915,0.006719127,-0.43911476,-0.33205529,0.14507333,-0.09539721,202109
29251231,2.18594439,-2.3610599,-4.0843619,0.293105778,-0.84829599,0.42280651,0.41012326,0.154838667,0.64789813,-0.06473407,-0.126923411,-0.49600884,0.14795399,-0.73213869,-0.25622636,202109


### 6.3.2 Modelado, producción y scoring

In [ ]:
PARAM$trainingstrategy$validate <- c(202107)

PARAM$trainingstrategy$training <- c(
  202106, 202105, 202104, 202103, 202102, 202101,
  202012, 202011, 202010, 202009, 202008, 202007,
  202006, 202005
)

PARAM$trainingstrategy$training_pct <- 1.0


PARAM$trainingstrategy$positivos <- c( "BAJA+1", "BAJA+2")

In [ ]:
# seteo la clase01   1={BAJA+1, BAJA+2}   0={CONTINUA}
dataset_pca[, clase01 := ifelse( clase_ternaria %in% PARAM$trainingstrategy$positivos, 1, 0 )]

In [ ]:
# los campos en los que se entrena
campos_buenos <- copy( setdiff(
    colnames(dataset_pca), c("clase_ternaria","clase01","azar"))
)

In [ ]:
if( !require("lightgbm")) install.packages("lightgbm")
require("lightgbm")

In [ ]:
for(seed in PARAM$semilla_primigenia) {

  cat("Ejecutando con semilla:", seed, "\n")

  # fijar semilla
  set.seed(seed, kind = "L'Ecuyer-CMRG")

  # generar azar para undersampling
  dataset_pca[, azar := runif(nrow(dataset_pca))]

  # undersampling para TRAIN
  dataset_pca[, fold_train :=
            foto_mes %in% PARAM$trainingstrategy$training &
            (clase_ternaria %in% c("BAJA+1", "BAJA+2") |
             azar < PARAM$trainingstrategy$training_pct)
  ]

  # ---- TRAIN ----
  dtrain <- lgb.Dataset(
    data  = data.matrix(dataset_pca[fold_train == TRUE, campos_buenos, with = FALSE]),
    label = dataset_pca[fold_train == TRUE, clase01],
    free_raw_data = TRUE
  )

  # ---- VALIDATION ----
  dvalidate <- lgb.Dataset(
    data  = data.matrix(dataset_pca[foto_mes %in% PARAM$trainingstrategy$validate,
                                campos_buenos, with = FALSE]),
    label = dataset_pca[foto_mes %in% PARAM$trainingstrategy$validate, clase01],
    free_raw_data = TRUE
  )

  # ---- PARÁMETROS LGBM ----
  PARAM$lgbm$param_fijos <- list(
    objective = "binary",
    metric = "auc",
    first_metric_only = TRUE,
    boost_from_average = TRUE,
    feature_pre_filter = FALSE,
    verbosity = -100,
    force_row_wise = TRUE,
    seed = seed,
    max_bin = 31,
    feature_fraction = 0.5,
    num_leaves = 256,
    min_data_in_leaf = 107,
    learning_rate = 0.0063,
    max_depth = 17,
    min_sum_hessian_in_leaf = 0.8994,
    scale_pos_weight = 1.1414
  )

  # ---- FINAL TRAIN (SIN undersampling) ----
  PARAM$trainingstrategy$final_train <- c(
    202107,
    202106, 202105, 202104, 202103, 202102, 202101,
    202012, 202011, 202010, 202009, 202008, 202007,
    202006, 202005
  )

  dataset_pca[, fold_final_train := foto_mes %in% PARAM$trainingstrategy$final_train ]

  dfinal_train <- lgb.Dataset(
    data  = data.matrix(dataset_pca[fold_final_train == TRUE, campos_buenos, with = FALSE]),
    label = dataset_pca[fold_final_train == TRUE, clase01],
    free_raw_data = TRUE
  )

  cat("Tamaño final_train:", nrow(dataset_pca[fold_final_train == TRUE]), "\n")

  # ---- ENTRENAMIENTO FINAL ----
  final_model <- lgb.train(
    data  = dfinal_train,
    param = PARAM$lgbm$param_fijos,
    verbose = -100
  )

  cat("Modelo final entrenado con semilla:", seed, "\n")

  # grabo a disco el modelo en un formato para seres humanos ... ponele ...
  archivo_modelo <- paste0("modelo_", seed, ".txt")
  lgb.save(final_model, archivo_modelo)

  # IMPORTANCIA
  tb_importancia <- as.data.table(lgb.importance(final_model))
  archivo_importancia <- paste0("impo_", seed, ".txt")
  fwrite(tb_importancia, file = archivo_importancia, sep = "\t")

  cat("Archivos guardados para semilla:", seed, "\n")

  PARAM$trainingstrategy$future <- c(202109)

  dfuture <- dfuture_completo

  # aplico final_model a dfuture
  prediccion <- predict(
    final_model,
    data.matrix(dfuture[, campos_buenos, with = FALSE])
  )

  tb_prediccion <- dfuture[, list(numero_de_cliente)]
  tb_prediccion[, prob := prediccion]

  # archivo de predicción por semilla
  archivo_pred <- paste0("prediccion_", seed, ".txt")

  fwrite(
    tb_prediccion,
    file = archivo_pred,
    sep = "\t"
  )

  # ============================================================
  #   GENERACIÓN ARCHIVOS KAGGLE (por semilla)
  # ============================================================

  PARAM$kaggle$competencia <- "labo-i-2025-virtual-gerencial"
  PARAM$kaggle$cortes <- seq(800, 1300, by = 50)

  # ordeno por probabilidad descendente
  setorder(tb_prediccion, -prob)

  dir.create("kaggle", showWarnings = FALSE)

  for (envios in PARAM$kaggle$cortes) {

    tb_prediccion[, Predicted := 0L]
    tb_prediccion[1:envios, Predicted := 1L]

    archivo_kaggle <- paste0(
      "./kaggle/PCA",
      PARAM$experimento, "_",
      seed, "_",
      envios, ".csv"
    )

    fwrite(
      tb_prediccion[, list(numero_de_cliente, Predicted)],
      file = archivo_kaggle,
      sep = ","
    )

    # comando de subida a Kaggle
    comando <- "kaggle competitions submit"
    competencia <- paste("-c", PARAM$kaggle$competencia)
    arch <- paste("-f", archivo_kaggle)

    mensaje <- paste0(
      "-m 'envios=", envios,
      "  semilla=", seed,
      "'"
    )

    linea <- paste(comando, competencia, arch, mensaje)

    salida <- system(linea, intern = TRUE)
    cat(salida, "\n")
  }
}


Ejecutando con semilla: 819509 
Tamaño final_train: 214269 
Modelo final entrenado con semilla: 819509 
Archivos guardados para semilla: 819509 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
